In [1]:
import numpy as np
import pandas as pd
SMOOTH = 1e-6

In [2]:
def iou_numpy(outputs: np.array, labels: np.array):
    outputs = outputs.squeeze(1)
    
    intersection = (outputs & labels).sum((1, 2))
    union = (outputs | labels).sum((1, 2))
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)
    
    thresholded = np.ceil(np.clip(20 * (iou - 0.5), 0, 10)) / 10
    
    return thresholded

In [3]:
labeled = pd.read_csv('../data/interim/labeled.csv')
unlabeled = pd.read_csv('../data/interim/unlabeled.csv')

In [38]:
unlabeled.head()

,Position,content,index,description,target,profession,profession_desc
0,секретар,срочн треб секретар выплат ежедневн работа луч...,363107946,срочн треб секретар выплат ежедневн работа луч...,-1,NaN,NaN
1,персональн менеджер,оконч вуз разбира закон банковск сфер работа с...,363144355,NaN,-1,NaN,NaN
2,продавец консультант ecco тц сит молл,облада хорош коммуникативн навык теб нрав прод...,363340232,датск компан ecco 50 лет вход тройк миров прои...,-1,NaN,NaN
3,агент риэлтор помощник риэлтор,организовыва провод просмотр показ объект полн...,363383054,NaN,-1,NaN,NaN
4,специалист поддержк клиент,легк ориентируеш огромн поток нов информац обл...,363982236,NaN,-1,NaN,NaN


In [4]:
okz = pd.read_csv('../data/interim/okz.csv', sep=',')

In [5]:
okz.head()

,code,control_number,profession,description
0,111,6.0,руководител представител федеральн региональн ...,руководител представител федеральн региональн ...
1,1111,7.0,руководител федеральн региональн орган законод...,руководител представител федеральн региональн ...
2,1112,3.0,руководител представител федеральн региональн ...,руководител представител федеральн региональн ...
3,1113,7.0,руководител орган местн самоуправлен,руководител орган местн самоуправлен выполня ц...
4,1114,0.0,высш должностн лиц политическ обществен организац,высш должностн лиц политическ обществен органи...


In [25]:
def make_dict_iou(position, okz):
    d_iou = {}
    for i in range(len(okz)):
        position_array = np.array(position.split())
        okz_array = np.array(okz.description[i].split())
        print(iou_numpy(position_array.reshape(1, 1, len(position_array)), okz_array.reshape(1, 1, len(okz_array))))
        # d_iou[okz.code[i]] = iou_numpy(np.array(position.split()), np.array(okz.description[i].split()))
    return d_iou

In [35]:
okz.description[0]

'руководител представител федеральн региональн орган законодательн судебн исполнительн власт аппарат ин орган определя формулир консультир непосредствен реализ политик федеральн региональн местн орган власт общин такж обществен организац разрабатыва ратифицир изменя отменя закон обществен норм прав планир организ управля контролир оценива общ деятельн министерств ведомств традицион общин обществен организац выполня обязан работник дан мал групп обычн включа руководств участ работ законодательн орган административн совет правительств законодательн собран орган местн самоуправлен обществен организац служб государствен административн орган официальн комитет расследован вопрос каса интерес обществ цел затрагива интерес избирател разработк консультирован вопрос проведен государствен политик исполнен бюджет законодательств нормативн обеспечен установлен цел организац разработк утвержден оценк программ политик процедур реализац рекомендац анализ оценк утвержден документ информацион записок пр

In [39]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'torch._C'